TODO
- Normal working hours
- Efficiency within those hours (either in meetings or build work)
- Split between meetings and build
- D3 visualization: https://observablehq.com/@d3/calendar-view

In [ ]:
import requests
from ics import Calendar
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt\

In [ ]:
url = 'http://p01-calendars.icloud.com/published/2/R7z0NY0Ja-eATwYdxvL0Cj5_0suPQs9_NokawvmHwpTuh04vObEvNZuL3-mYKubUqej19L4ZXAETnqZqlGuUb0bkBON4r9c7vUIHp7Ba2S0'
raw = requests.get(url).text
c = Calendar(raw.replace('BEGIN:VCALENDAR', 'BEGIN:VCALENDAR\r\nPRODID:noah-rocks'))
c

In [ ]:
items = []
for e in list(c.events):
    items.append({
        'name': e.name,
        'begin': str(e.begin),
        'end': str(e.end),
        'duration_min': (e.duration.total_seconds())/60.0,
        'duration_hr': (e.duration.total_seconds())/60.0/60.0
    })

df = pd.DataFrame(items)
df['begin'] = pd.to_datetime(df.begin)
df['begin'] = df.begin.apply(lambda x: x.replace(tzinfo=None))
df['date'] = df.begin.dt.date
df['end'] = pd.to_datetime(df.end)
df['end'] = df.begin.apply(lambda x: x.replace(tzinfo=None))
df.head()

In [ ]:
df.sort_values(by='date', ascending=False)

In [ ]:
def flip_category(s):
    return ' '.join(s.split(' ')[-1:] + s.split(' ')[:-1])

def sanitize_name(s):
    s_lower = s.lower()
    if ('forward' in s_lower) & ('build' in s_lower):
        return 'Forward: Build'
    if ('forward' in s_lower) & ('meeting' in s_lower):
        return 'Forward: Meeting'
    return s

df['name'] = df['name'].apply(flip_category)
df['name'] = df['name'].apply(sanitize_name)
df.head()

In [ ]:
import matplotlib.dates as mdates

In [ ]:
def plot_within_range(df_in, start, end=None):
    if end is None:
        end = '2021-01-01'
    df1 = df_in[
        (df_in['date'] >= pd.to_datetime(start)) &\
        (df_in['date'] < pd.to_datetime(end))
    ].copy()
    dft = df1.groupby(['date', 'name'])['duration_hr'].agg(['sum']).reset_index()
    dft.head()

    my_dates = pd.date_range(dft.date.min(), dft.date.max(), freq='D')
    idx = pd.MultiIndex.from_product([my_dates, dft.name.unique()], names=['date', 'name'])
    dft = dft.set_index(['date', 'name']).reindex(idx).fillna(0).reset_index()
    dft.sort_values(by='date', ascending=False)
    dft['date'] = dft['date'].apply(lambda x: x.strftime('%m/%d/%Y'))

    dft.pivot(index='date', columns='name', values='sum').plot.bar(stacked=True);
    plt.xlabel('');
    plt.ylabel('Productive Hours');

    ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0], reverse=True))
    ax.legend(handles, labels)
    plt.show()
    return

# For grid view https://observablehq.com/@d3/calendar-view
plt.rcParams['figure.figsize'] = [12, 8]
plot_within_range(df, '2020-04-17')

In [ ]:
plt.rcParams['figure.figsize'] = [10, 20]
sns.barplot(y=df.name.value_counts().index, x=df.name.value_counts());